#### Step 1: Split Index Code into different columns

index_overall,index_main_type,index_service_type,index_region

#### Step 2: Remove Index Code from Item Code

eg. 0101010101001 to 001

#### Step 3: Transform structure of df from cols 
[index_overall,index_main_type,index_service_type,index_region,item_code,[months]] 
to 
[index_overall,index_main_type,index_service_type,index_region,item_code,month,price]

In [2]:
import pandas as pd

In [13]:
file_path = "data2.csv"
# df = pd.DataFrame({'Index Code': ['0101010101', '0101010102', '0101010203', '0101010204', '0102010203'],
#                    'Item Code': ['0101010101001', '0101010102002', '0101010203003', '0101010204004', '0102010203005'],
#                    'Jun 2022': [1, 2, 3, 4, 5],
#                    'Jul 2022': [2, 3, 4, 5, 6],
#                    'Aug 2022': [3, 4, 5, 6, 7],
#                    'Sep 2022': [4, 5, 6, 7, 8],
#                    'Oct 2022': [5, 6, 7, 8, 9],
#                    'Nov 2022': [6, 7, 8, 9, 10],
#                    'Dec 2022': [7, 8, 9, 10, 11],
#                    'Jan 2023': [8, 9, 10, 11, 12],
#                    })

# df.to_csv(file_path, index=False)

#read csv, and keep Index Code and Item Code as string
df = pd.read_csv(file_path, dtype={'Index Code': str, 'Item Code': str})

#Make months into rows
df = df.melt(id_vars=["Index Code", "Item Code"], 
                        var_name="month", 
                        value_name="price")

for index,row in df.iterrows():
    ## Step 1: Split Index Code into different columns
    #index_overall,index_main_type,index_freight_type,index_service_type,index_region
    df.loc[index,'index_overall'] = row['Index Code'][:2]
    df.loc[index,'index_main_type'] = row['Index Code'][2:4]
    df.loc[index,'index_freight_type'] = row['Index Code'][4:6]
    df.loc[index,'index_service_type'] = row['Index Code'][6:8]
    df.loc[index,'index_region'] = row['Index Code'][8:10]

    ## Step 2: Check if Index Code is in Item Code
    if row['Index Code'] in row['Item Code']:
        #remove Index Code from Item Code
        df.loc[index,'item_code'] = row['Item Code'].replace(row['Index Code'],'')

    # convert month to date, 1st day of the month
    df.loc[index,'month'] = pd.to_datetime(row['month'], format='%b %Y').strftime('%Y-%m-%d')

df.drop(columns=['Index Code','Item Code'], inplace=True)
#re-order columns
df = df[['index_overall', 'index_main_type', 'index_freight_type', 'index_service_type', 'index_region', 'item_code', 'month', 'price']]
df.to_csv('output.csv', index=False)